In [8]:
import numpy as np
from copy import deepcopy

In [9]:
def filled(matrix):
    c = 0
    for i in matrix:
        for j in i:
            if j != 0:
                c+=1
    return c


def areIdentical(matrixOne, matrixTwo):
    # small conditions for the sake of complexity(code runs way faster now)
    # basically why to bother checking and comparing element of both if even filled are not same 
    if matrixOne[1][1] == matrixTwo[1][1] and filled(matrixOne) == filled(matrixTwo):
        return np.array_equal(matrixOne, matrixTwo) or np.array_equal(np.flipud(matrixOne), matrixTwo) or np.array_equal(np.fliplr(matrixOne), matrixTwo) \
            or np.array_equal(np.rot90(matrixOne), matrixTwo) or np.array_equal(np.flipud(np.rot90(matrixOne)), matrixTwo) or np.array_equal(np.fliplr(np.rot90(matrixOne)), matrixTwo) \
            or np.array_equal(np.rot90(matrixOne, 2), matrixTwo) or np.array_equal(np.flipud(np.rot90(matrixOne, 2)), matrixTwo) or np.array_equal(np.fliplr(np.rot90(matrixOne, 2)), matrixTwo) \
            or np.array_equal(np.rot90(matrixOne,3), matrixTwo) or np.array_equal(np.flipud(np.rot90(matrixOne,3)), matrixTwo) or np.array_equal(np.fliplr(np.rot90(matrixOne,3)), matrixTwo)
    else:
        return False

def isWon(matrix):
    for i in range(3):
        if matrix[0][i] != 0 and matrix[0][i] == matrix[1][i] == matrix[2][i]:
            return matrix[0][i]
        if matrix[i][0] != 0 and matrix[i][0] == matrix[i][1] == matrix[i][2]:
            return matrix[i][0]
    if matrix[0][0] != 0 and matrix[0][0] == matrix[1][1] == matrix[2][2]:
        return matrix[0][0]
    if matrix[0][2] != 0 and matrix[0][2] == matrix[1][1] == matrix[2][0]:
        return matrix[0][2]
    return 0

In [10]:
def getStates(matrix, isOurMove):
    moves = []
    for i in range(3):
        for j in range(3):
            if matrix[i][j] == 0:
                newMatrix = deepcopy(matrix)
                newMatrix[i][j] = 1 if isOurMove else 2
                moves.append(newMatrix)
    return moves

def createStates(matrix, isOurMove=True, states = []):
    if filled(matrix) == 9:
        return states
    newStates = getStates(matrix, isOurMove)
    for i in newStates:
        found = False
        for j in states:
            if areIdentical(i,j):
                found = True
                break
        if not found and not isWon(i) and filled(i) < 8:
            if isOurMove:
                states.append(i)
            createStates(i, isOurMove=not isOurMove, states = states)
    return states

In [164]:
# RUN FOR FIRST TIME AND STORE IN CACHE FOR NEXT TIMES(Finding 304 unique matrixes take time)
z = createStates(np.zeros((3,3)), False, [np.zeros((3,3))])
print('you\'re done')
%store z

Stored 'z' (list)


In [4]:
# RUN THIS FOR NEXT TIME SO YOU DON"T NEED TO DO COMPUTATION AGAIN
%store -r z

In [5]:
len(z) # 304!!! WE'RE ON TRACK!!!
# for i in z:
#     if filled(i) == 8:
#         displayGame(i)

304

In [45]:
"""
https://www.mscroggs.co.uk/blog/19
Initially, MENACE begins with four beads of each colour in the first move box, 
three in the third move boxes, two in the fifth move boxes and one in the final move boxes. 
Removing one bead from each box on losing means that later moves are more heavily discouraged. 
This helps MENACE learn more quickly, as the later moves are more likely to have led to the loss.
"""

def displayGame(board):
    # strange CLI asesthetics
    print('\n-------')
    for i in board:
        for j in i:
            if j == 1:
                print('O', end='|')
            elif j == 2:
                print('X', end='|')
            else:
                print('_', end='|')
        print('\n-------')

# putting intial matches in all. 4 of each unfilled place
def getInitalMatches(z):
    initialNumber = 4
    moves = list(range(9))
    allMatches = []
    for i in z:
        emptyMoves = []
        for _i in range(3):
            for _j in range(3):
                if i[_i][_j] == 0:
                    emptyMoves.append(_i * 3 + _j)
        allMatches.append({
            'matrix': i,
            'places': emptyMoves*initialNumber
        })
    return allMatches

def getMatchbox(board, matchBoxes):
    for i in matchBoxes:
        if areIdentical(board, i['matrix']):
            return i

def findEmptyPlaces(board):
    free = []
    for i in range(3):
        for j in range(3):
            if board[i][j] == 0:
                free.append(3*i+j)
    return free
initialMatches = getInitalMatches(z)
len(initialMatches)

304

In [41]:
np.random.choice(findEmptyPlaces([[0,1,0], [1,0,1], [0,0,1]]))


3

In [60]:
_initialMatches = deepcopy(initialMatches)
def trainMenace(iters, _initialMatches, showPrints, userInput = False):   
    for _ in range(iters):
        board = np.zeros((3,3))
        playerMove = False
        # array to keep track of menace moves so we can update at the end
        menaceMoves = []
        while filled(board) != 9 and not isWon(board):
            if playerMove:
                # player move logic
                inp = 0
                if userInput:
                    inp = int(input('your move:')) - 1 # well we start from 1 in head so....
                else:
                    inp = np.random.choice(findEmptyPlaces(board))
                board[inp//3][inp%3] = 2
            else:
                if filled(board) < 8:
                    # THE MAGIC PART LOL
                    # find appropriate matchbox from menace
                    matchBox = getMatchbox(board, _initialMatches)
                    if matchBox:
                        # SO WE WILL ROTATE/FLIP the matrix if needed to make it easier for menace to choose
                        board = deepcopy(matchBox['matrix'])
                        choice = np.random.choice(matchBox['places'])
                        menaceMoves.append({
                            'matrix': matchBox['matrix'],
                            'choice': choice
                        })
                        if showPrints:
                            print(f'menace played {choice+1}')
                        # updating the board
                        board[choice//3][choice%3] = 1
                else:
                    # automatically fill last choice for menace
                    for i in range(3):
                        for j in range(3):
                            if board[i][j] == 0:
                                if showPrints:
                                    print(f'menace played {i*3+j+1}')
                                board[i][j] = 1
            if showPrints:
                displayGame(board)
            playerMove = not playerMove

        winningState = isWon(board)
        if showPrints:
            if not winningState:
                print('DRAW SO MANACE WILL ADD 1 MATCH IN ALL')
            elif winningState == 1:
                print('MENACE WON SO WILL ADD 2 MATCH IN ALL')
            else:
                print('MENACE LOST SO WILL REMOVE 1 MATCH FROM ALL')
        # THE MOST MAGIC PART, CHANGING PROBABILITIES DEPENDING UPON WINNING LOSING AND ALL
        for i in menaceMoves:
            for j in _initialMatches:
                if np.array_equal(i['matrix'], j['matrix']):
                    if not winningState:
                        # if draw add 1
                        j['places'].append(i['choice'])
                    elif winningState == 1:
                        # if won add twice
                        j['places'].extend([i['choice']]*2)
                    else:
                        # if lost remove once
                        j['places'].remove(i['choice'])
#                     print(j['places'])
                    break
    return _initialMatches
trained = trainMenace(10000, _initialMatches, False, False)
print('done training')

done training


In [61]:
trainMenace(1, trained, True, True)
print('aye')

menace played 7

-------
_|_|_|
-------
_|_|_|
-------
O|_|_|
-------


your move: 1



-------
X|_|_|
-------
_|_|_|
-------
O|_|_|
-------
menace played 4

-------
X|_|O|
-------
O|_|_|
-------
_|_|_|
-------


your move: 5



-------
X|_|O|
-------
O|X|_|
-------
_|_|_|
-------
menace played 8

-------
X|O|_|
-------
_|X|_|
-------
O|O|_|
-------


your move: 9



-------
X|O|_|
-------
_|X|_|
-------
O|O|X|
-------
MENACE LOST SO WILL REMOVE 1 MATCH FROM ALL
aye


In [62]:
trained

[{'matrix': array([[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]),
  'places': [1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   8,
   8,
   6,
   6,
   6,
   6,
   6,
   6,
   4,
   4,
   4,
   4,
   4,
   4,
   5,
   5,
   5,
   5,
   6,
   6,
   4,
   4,
   6,
   6,
   8,
   8,
   6,
   6,
   4,
   4,
   6,
   6,
   4,
   4,
   6,
   6,
   4,
   4,
   6,
   6,
   4,
   4,
   4,
   4,
   1,
   1,
   2,
   4,
   4,
   2,
   2,
   4,
   4,
   2,
   2,
   6,
   6,
   6,
   6,
   6,
   2,
   2,
   2,
   2,
   5,
   5,
   2,
   2,
   4,
   4,
   6,
   6,
   4,
   4,
   2,
   2,
   3,
   2,
   2,
   6,
   6,
   2,
   2,
   6,
   6,
   6,
   6,
   2,
   2,
   6,
   6,
   6,
   6,
   6,
   2,
   6,
   6,
   2,
   2,
   4,
   4,
   6,
   6,
   5,
   6,
   6,
   6,
   6,
   6,
   6,
   2,
   2,
   4,
   4,
   2,
   2,
   2,
   2,
   6,
   